# Example of how to use cira with sci-kit learn 

understand that this is just an example and not a good model for actual trading.

In this example we will setup cira and use Liner Regression as an model and try to make prediction on a crypto asset. 

### Cira setup

In [19]:
import cira

In [20]:
cira.auth.KEY_FILE = "../alpc_key.json"
assert cira.auth.check_keys(), "the set keys dose not work"

In [21]:
portfolio = cira.Portfolio()
crypto_exchange = cira.CryptoExchange()

### Fetching data 

In [22]:
from datetime import datetime

SYMBOL = "ETH/USD"

cry = cira.Cryptocurrency(SYMBOL)    

start = datetime(2020, 7, 1)
end = datetime(2023, 7, 1)

data = cry.historical_data_df(start_date=start, end_date=end)
data.head()

open    high     low   close   volume  \
symbol  timestamp                                                            
ETH/USD 2020-07-01 05:00:00+00:00  225.60  232.42  224.93  231.33  180.019   
        2020-07-02 05:00:00+00:00  231.22  231.66  223.16  227.15  791.799   
        2020-07-03 05:00:00+00:00  227.00  228.07  224.98  226.10  167.520   
        2020-07-04 05:00:00+00:00  226.59  230.38  226.00  228.26   89.578   
        2020-07-05 05:00:00+00:00  228.19  228.19  223.79  227.96   66.322   

                                   trade_count        vwap  
symbol  timestamp                                           
ETH/USD 2020-07-01 05:00:00+00:00         91.0  229.234661  
        2020-07-02 05:00:00+00:00        106.0  226.154453  
        2020-07-03 05:00:00+00:00        109.0  226.695652  
        2020-07-04 05:00:00+00:00         57.0  228.061196  
        2020-07-05 05:00:00+00:00         27.0  225.873532

### Machine learning

training the model 

In [23]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error


y = data[['close']]
X = data[['open', 'high', 'low']]

window_size = 10
y_preds = []
actual_prices = []

for i in range(len(X) - window_size):
    X_window = X.iloc[i:i + window_size]
    y_window = y.iloc[i:i + window_size]
    
    model = LinearRegression()
    model.fit(X_window.to_numpy(), y_window.to_numpy())
    
    next_day_features = X.iloc[i + window_size]
    next_day_pred = model.predict([next_day_features])[0]
    
    y_preds.append(next_day_pred)
    actual_prices.append(y.iloc[i + window_size])

mse = mean_squared_error(actual_prices, y_preds)
print(f"Mean Squared Error: {mse}")

Mean Squared Error: 3090.84756905199


using the models predictions

In [24]:
def run(y_preds:list, cash:float, shares:int) -> float:
    for i in range(len(y_preds) - 1):
        if y_preds[i + 1] > y_preds[i] and cash >= y_preds[i + 1]: 
            cash -= y_preds[i + 1]
            shares += 1
            #print(f"Buy 1 share at price: {y_preds[i + 1]}")
        
        elif y_preds[i + 1] < y_preds[i] and shares > 0: 
            cash += y_preds[i + 1]
            shares -= 1
            #print(f"Sell 1 share at price: {y_preds[i + 1]}")

    final_cash = cash + shares * y_preds[-1]
    return final_cash[0]

In [25]:
cash = portfolio.cash()  # Initial capital
shares = portfolio.position_in(str(cry)).quantity()  # Number of shares held
print("predicted profit/loss:", f"{run(y_preds, cash, shares) - portfolio.cash():.1f} USD$")

predicted profit/loss: 29361.5 USD$
